# **BDA - Phase 1**

**MySQL**

### **1. Load the data**

In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine, text
import time

In [2]:
host_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/host.csv',
    dtype={
        'id': 'int64',
        'url': 'string',
        'name': 'string',
        'about': 'string',
        'is_superhost': 'bool',
        'thumbnail_url': 'string',
        'picture_url': 'string',
        'verifications': 'string',
        'location': 'string',
        'neighbourhood': 'string'
    }
)

host_statistics_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/host_statistics.csv',
    dtype={
        "id": "int32",
        "host_id": "int32",
        "response_time": "string",
        "response_rate": "float32",
        "acceptance_rate": "float32",
        "listings_count": "int16",
        "total_listings_count": "int16",
        "calculated_host_listings_count": "int16",
        "calculated_host_listings_count_entire_homes": "int16",
        "calculated_host_listings_count_private_rooms": "int16",
        "calculated_host_listings_count_shared_rooms": "int16"
    },
    parse_dates=["since"]
)

listings_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/listings.csv', 
    dtype={
        "id": "int32",
        "listing_url": "string",
        "name": "string",
        "description": "string",
        "neighborhood_overview": "string",
        "picture_url": "string",
        "host_id": "int32",
        "neighbourhood": "string",
        "latitude": "float32",
        "longitude": "float32",
        "property_type": "string",
        "room_type": "string",
        "accommodates": "int16",
        "bathrooms": "float32",
        "bathrooms_text": "string",
        "bedrooms": "float32",
        "beds": "float32",
        "amenities": "string",
        "base_price": "float32",
        "minimum_nights": "int16",
        "maximum_nights": "int16",
        "has_availability": "bool",
        "instant_bookable": "bool"
    },
    parse_dates=["last_scraped"]
)

calendar_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/calendar.csv',
    dtype={
        "id": "int32",
        "listing_id": "int32",
        "available": "bool",
        "price": "float32"
    },
    parse_dates=["date"]
)

reviews_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/reviews.csv',
    dtype={
        "id": "int32",
        "listing_id": "int32",
        "reviewer_id": "int32",
        "reviewer_name": "string",
        "comments": "string"
    },
    parse_dates=["date"]
)

availability_statistics_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/availability_statistics.csv',
    dtype={
        "id": "int32",
        "listing_id": "int32",
        "minimum_minimum_nights": "int16",
        "maximum_minimum_nights": "int16",
        "minimum_maximum_nights": "int16",
        "maximum_maximum_nights": "int16",
        "minimum_nights_avg_ntm": "float32",
        "maximum_nights_avg_ntm": "float32",
        "availability_30": "int16",
        "availability_60": "int16",
        "availability_90": "int16",
        "availability_365": "int16"
    }
)

reviews_statistics_df = pd.read_csv(
    filepath_or_buffer='../data/clean-data/relational/reviews_statistics.csv',
    dtype={
        "id": "int32",
        "listing_id": "int32",
        "number_of_reviews": "int16",
        "number_of_reviews_ltm": "int16",
        "number_of_reviews_l30d": "int16",
        "review_scores_rating": "float32",
        "review_scores_accuracy": "float32",
        "review_scores_cleanliness": "float32",
        "review_scores_checkin": "float32",
        "review_scores_communication": "float32",
        "review_scores_location": "float32",
        "review_scores_value": "float32",
        "reviews_per_month": "float32"
    },
    parse_dates=["first_review", "last_review"]
)

### **2. Create the DB**

In [3]:
username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'
schema_file = '../schemas/db_schema.sql'

def load_sql_schema(cursor, schema_file):
    try:
        with open(schema_file, 'r') as f:
            sql_script = f.read()
        statements = sql_script.split(';')
        for statement in statements:
            if statement.strip():
                try:
                    cursor.execute(statement)
                    print(f"Executed: {statement.strip().splitlines()[0]}...")
                except Exception as e:
                    print(f"Error with statement: {statement.strip().splitlines()[0]}...\n{e}")
        print("Schema successfully loaded into the database!")
    except FileNotFoundError:
        print(f"Schema file not found: {schema_file}")
        raise
    except Exception as e:
        print(f"Error reading schema file: {e}")
        raise

def load_dataframes_to_mysql(engine, dataframes):
    try:
        for table_name, dataframe in dataframes.items():
            dataframe.to_sql(table_name, con=engine, if_exists='replace', index=False, method='multi')
            print(f"Table '{table_name}' data inserted successfully!")
    except Exception as e:
        print(f"Error loading dataframes: {e}")
        raise

# Main execution
try:
    with mysql.connector.connect(
        host=host,
        user=username,
        password=password
    ) as mydb:
        with mydb.cursor() as cursor:
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
            cursor.execute(f"USE {database_name}")
            load_sql_schema(cursor, schema_file)
        mydb.commit()

    db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
    engine = create_engine(db_url)

    # dataframes = {
    #     "host_statistics": host_statistics_df,
    #     "calendar": calendar_df,
    #     "reviews": reviews_df,
    #     "availability_statistics": availability_statistics_df,
    #     "reviews_statistics": reviews_statistics_df,
    #     "listings": listings_df,
    #     "host": host_df,
    # }

    dataframes = {
        "HOST_STATISTICS": host_statistics_df,
        "CALENDAR": calendar_df,
        "REVIEWS": reviews_df,
        "AVAILABILITY_STATISTICS": availability_statistics_df,
        "REVIEWS_STATISTICS": reviews_statistics_df,
        "LISTINGS": listings_df,
        "HOST": host_df,
    }

    load_dataframes_to_mysql(engine, dataframes)

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    print("Process completed.")

Executed: -- Create the HOST table...
Executed: -- Create the HOST_STATISTICS table...
Executed: -- Create the LISTINGS table...
Executed: -- Create the CALENDAR table...
Executed: -- Create the REVIEWS table...
Executed: -- Create the AVAILABILITY_STATISTICS table...
Executed: -- Create the REVIEWS_STATISTICS table...
Error with statement: CREATE TABLE RESULTS (...
1050 (42S01): Table 'results' already exists
Schema successfully loaded into the database!
Table 'HOST_STATISTICS' data inserted successfully!
Table 'CALENDAR' data inserted successfully!
Table 'REVIEWS' data inserted successfully!
Table 'AVAILABILITY_STATISTICS' data inserted successfully!
Table 'REVIEWS_STATISTICS' data inserted successfully!
Table 'LISTINGS' data inserted successfully!
Table 'HOST' data inserted successfully!
Process completed.


### **3. Queries**

**Simple Queries:**

In [74]:
username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

simple_query = """
SELECT *
FROM listings
WHERE has_availability = TRUE
AND bedrooms = 2
AND base_price < 100;
"""

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

try:
    with engine.connect() as connection:
        start_time = time.time()
        result = connection.execute(text(simple_query))
        results = result.fetchall()
        end_time = time.time()
        
        print(f"{len(results)} results found in {end_time - start_time:.4f} seconds")
except Exception as e:
    print(f"An error occurred while executing the query: {e}")

17 results found in 0.0017 seconds


In [75]:
username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

simple_query = """
WITH unique_host_statistics AS (
    SELECT host_id, MAX(listings_count) AS listings_count
    FROM host_statistics
    WHERE listings_count > 1
    GROUP BY host_id
)
SELECT h.id, h.location
FROM host h
JOIN unique_host_statistics uhs ON h.id = uhs.host_id
WHERE h.location = 'Albany, NY';
"""

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)


try:
    with engine.connect() as connection:
        start_time = time.time()
        result = connection.execute(text(simple_query))
        results = result.fetchall()
        end_time = time.time()

        print(f"{len(results)} results found in {end_time - start_time:.4f} seconds")
except Exception as e:
    print(f"An error occurred while executing the query: {e}")

262 results found in 0.0025 seconds


### **3.1 Complex Queries:**

**Complex Query 1: Hosts em New York**

In [4]:
complex_query = """
WITH average_prices AS (
    SELECT 
        l.host_id,
        AVG(l.base_price) AS avg_price
    FROM listings l
    WHERE l.has_availability = TRUE
    GROUP BY l.host_id
),
host_with_high_listings AS (
    SELECT 
        l.host_id,
        COUNT(DISTINCT l.id) AS listings_count
    FROM listings l
    GROUP BY l.host_id
    HAVING COUNT(DISTINCT l.id) > 5
)
SELECT 
    h.id AS host_id,
    h.name AS host_name,
    h.location AS host_location,
    apl.avg_price AS average_price,
    hhl.listings_count AS total_listings
FROM host h
JOIN average_prices apl ON h.id = apl.host_id
JOIN host_with_high_listings hhl ON h.id = hhl.host_id
WHERE apl.avg_price < 150
AND h.location = 'New York, NY'
ORDER BY apl.avg_price ASC;
"""

insert_query = """
INSERT INTO RESULTS (host_id, host_name, host_location, average_price, total_listings)
VALUES (:host_id, :host_name, :host_location, :average_price, :total_listings)
"""

In [5]:
# without index

try:
    with engine.connect() as connection:
        start_time = time.time()

        result = connection.execute(text(complex_query))
        results = result.fetchall()

        if results:
            for row in results:
                connection.execute(
                    text(insert_query),
                    {
                        "host_id": row[0],
                        "host_name": row[1],
                        "host_location": row[2],
                        "average_price": row[3],
                        "total_listings": row[4],
                    }
                )
            
            connection.commit()
            print(f"{len(results)} records inserted into RESULTS.")

            end_time = time.time()
            print(f"{len(results)} results found and inserted in {end_time - start_time:.4f} seconds")
        else:
            print("No results to insert.")

except Exception as e:
    print(f"An error occurred: {e}")

6 records inserted into RESULTS.
6 results found and inserted in 0.0066 seconds


In [6]:
# with indexes and insert functionality

try:
    with engine.connect() as connection:
        print("Existing indexes")
        listings_indexes = connection.execute(text("SHOW INDEX FROM LISTINGS"))
        host_statistics_indexes = connection.execute(text("SHOW INDEX FROM HOST_STATISTICS"))
        host_indexes = connection.execute(text("SHOW INDEX FROM HOST"))
        
        if listings_indexes.rowcount > 0:
            print("\nIndexes in LISTINGS:")
            for row in listings_indexes:
                print(row)
        else:
            print("No indexes in LISTINGS")

        if host_statistics_indexes.rowcount > 0:
            print("\nIndexes in HOST_STATISTICS:")
            for row in host_statistics_indexes:
                print(row)
        else:
            print("No indexes in HOST_STATISTICS")

        if host_indexes.rowcount > 0:
            print("\nIndexes in HOST:")
            for row in host_indexes:
                print(row)
        else:
            print("No indexes in HOST")

        print("\nCreating simple indexes")
        connection.execute(text("CREATE INDEX idx_listings_host_id ON LISTINGS (host_id);")) 
        connection.execute(text("CREATE INDEX idx_host_statistics_host_id ON HOST_STATISTICS (host_id);")) 
        connection.execute(text("CREATE INDEX idx_host_location ON HOST (location(255));")) 
        print("Simple Indexes created successfully.")

        print("\nCreating compound indexes")
        connection.execute(text("CREATE INDEX idx_listings_availability_price ON LISTINGS (has_availability, base_price);")) 
        connection.execute(text("CREATE INDEX idx_host_statistics_listings_count ON HOST_STATISTICS (listings_count);"))
        print("Compound Indexes created successfully")
        
        print("\nExecuting the query")
        start_time = time.time()
        result = connection.execute(text(complex_query))
        results = result.fetchall()

        if results:
            for row in results:
                connection.execute(
                    text(insert_query),
                    {
                        "host_id": row[0],
                        "host_name": row[1],
                        "host_location": row[2],
                        "average_price": row[3],
                        "total_listings": row[4],
                    }
                )
            print(f"{len(results)} records inserted into RESULTS.")

            end_time = time.time()
            print(f"{len(results)} results found and inserted in {end_time - start_time:.4f} seconds")
        else:
            print("No results to insert.")


        print("\nRemoving created indexes")
        connection.execute(text("DROP INDEX idx_listings_host_id ON LISTINGS;"))
        connection.execute(text("DROP INDEX idx_host_statistics_host_id ON HOST_STATISTICS;"))
        connection.execute(text("DROP INDEX idx_host_location ON HOST;"))
        connection.execute(text("DROP INDEX idx_listings_availability_price ON LISTINGS;"))
        connection.execute(text("DROP INDEX idx_host_statistics_listings_count ON HOST_STATISTICS;"))
        print("Indexes removed successfully.")
except Exception as e:
    print(f"An error occurred while creating the indexes or executing the query: {e}")

Existing indexes
No indexes in LISTINGS
No indexes in HOST_STATISTICS
No indexes in HOST

Creating simple indexes
Simple Indexes created successfully.

Creating compound indexes
Compound Indexes created successfully

Executing the query
6 records inserted into RESULTS.
6 results found and inserted in 0.0031 seconds

Removing created indexes
Indexes removed successfully.


In [97]:
# visualize insert in RESULTS table

username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

try:
    with engine.connect() as connection:
        query_all = text("SELECT * FROM RESULTS")
        result_all = connection.execute(query_all)
        rows = result_all.fetchall()
        
        print("Contents of the RESULTS table:")
        for row in rows:
            print(row)
        
        query_count = text("SELECT COUNT(*) FROM RESULTS;")
        result_count = connection.execute(query_count)
        count = result_count.scalar()  # single value result
        
        print(f"The RESULTS table contains {count} rows.")

except Exception as e:
    print(f"An error occurred: {e}")

Contents of the RESULTS table:
(109, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(110, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(111, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(112, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(113, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(114, 42708277, 'Rodney', 'New York, NY', 102.167, 8)
(115, 110453341, 'Alexis', 'New York, NY', 103.0, 7)
(116, 385664127, 'Dillon', 'New York, NY', 104.5, 7)
(117, 385664127, 'Dillon', 'New York, NY', 104.5, 7)
The RESULTS table contains 9 rows.


In [94]:
# clear RESULTS table

try:
    with engine.connect() as connection:
        query_clear = text("DELETE FROM RESULTS")
        result_clear = connection.execute(query_clear)
    
        connection.commit()    
        print(f"All rows deleted from the RESULTS table. Total rows affected: {result_clear.rowcount}")

except Exception as e:
    print(f"An error occurred while clearing the RESULTS table: {e}")

All rows deleted from the RESULTS table. Total rows affected: 9


**Complex query 2: Atualização de preços de Listings**

In [81]:
base_price_update_query = """
WITH high_reviewed_listings AS (
    SELECT 
        rs.listing_id
    FROM REVIEWS_STATISTICS rs
    WHERE rs.review_scores_rating > 4.5
      AND rs.number_of_reviews > 100
),
active_hosts AS (
    SELECT 
        hs.host_id,
        MAX(hs.total_listings_count) AS max_listings_count,
        SUM(l.has_availability) AS available_listings
    FROM HOST_STATISTICS hs
    JOIN LISTINGS l ON hs.host_id = l.host_id
    GROUP BY hs.host_id
)
UPDATE LISTINGS l
JOIN high_reviewed_listings hl ON l.id = hl.listing_id
JOIN active_hosts ah ON l.host_id = ah.host_id
JOIN HOST h ON l.host_id = h.id
SET l.base_price = l.base_price * 1.1 
WHERE ah.max_listings_count > 5
    AND ah.available_listings > 3
    AND l.base_price < 300
    AND h.location = 'Albany, NY'
"""

In [82]:
# without index

username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

try:
    with engine.connect() as connection:
        start_time = time.time()
        result = connection.execute(text(base_price_update_query))
        connection.commit()
        end_time = time.time()
        
        affected_rows = result.rowcount
        print(f"{affected_rows} results changed in {end_time - start_time:.4f} seconds")
except Exception as e:
    print(f"An error occurred while executing the query: {e}")

24 results changed in 0.2389 seconds


In [83]:
# with indexes

try:
    with engine.connect() as connection:
        print("Existing indexes")
        listings_indexes = connection.execute(text("SHOW INDEX FROM LISTINGS"))
        reviews_statistics_indexes = connection.execute(text("SHOW INDEX FROM REVIEWS_STATISTICS"))

        if listings_indexes.rowcount > 0:
            print("\nIndexes in LISTINGS:")
            for row in listings_indexes:
                print(row)
        else:
            print("No indexes in LISTINGS")

        if reviews_statistics_indexes.rowcount > 0:
            print("\nIndexes in REVIEWS_STATISTICS:")
            for row in reviews_statistics_indexes:
                print(row)
        else:
            print("No indexes in REVIEWS_STATISTICS")

        # add indexes
        print("\nCreating indexes")
        connection.execute(text("CREATE INDEX id_index ON LISTINGS (id);"))
        connection.execute(text("CREATE INDEX RS_index ON REVIEWS_STATISTICS (listing_id);"))
        connection.execute(text("CREATE INDEX list_index ON LISTINGS (base_price, has_availability);"))
        connection.execute(text("CREATE INDEX Where_index ON REVIEWS_STATISTICS (number_of_reviews, review_scores_rating);"))
        print("Indexes created successfully.")

        listings_indexes = connection.execute(text("SHOW INDEX FROM LISTINGS"))
        reviews_statistics_indexes = connection.execute(text("SHOW INDEX FROM REVIEWS_STATISTICS"))

        print("Indexes in LISTINGS:")
        for row in listings_indexes:
            print(row)

        print("Indexes in REVIEWS_STATISTICS:")
        for row in reviews_statistics_indexes:
            print(row)

        # execute the query
        print("\nExecuting the query")
        start_time = time.time()
        result = connection.execute(text(base_price_update_query))
        connection.commit()
        end_time = time.time()
        
        affected_rows = result.rowcount
        print(f"{affected_rows} results changed in {end_time - start_time:.4f} seconds")

        # remove indexes
        print("\nRemoving created indexes")
        connection.execute(text("DROP INDEX id_index ON LISTINGS;"))
        connection.execute(text("DROP INDEX RS_index ON REVIEWS_STATISTICS;"))
        connection.execute(text("DROP INDEX list_index ON LISTINGS;"))
        connection.execute(text("DROP INDEX Where_index ON REVIEWS_STATISTICS;"))
        print("Indexes removed successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

Existing indexes
No indexes in LISTINGS
No indexes in REVIEWS_STATISTICS

Creating indexes
Indexes created successfully.
Indexes in LISTINGS:
('LISTINGS', 1, 'id_index', 1, 'id', 'A', 371, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('LISTINGS', 1, 'list_index', 1, 'base_price', 'A', 170, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('LISTINGS', 1, 'list_index', 2, 'has_availability', 'A', 174, None, None, 'YES', 'BTREE', '', '', 'YES', None)
Indexes in REVIEWS_STATISTICS:
('REVIEWS_STATISTICS', 1, 'RS_index', 1, 'listing_id', 'A', 426, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('REVIEWS_STATISTICS', 1, 'Where_index', 1, 'number_of_reviews', 'A', 140, None, None, 'YES', 'BTREE', '', '', 'YES', None)
('REVIEWS_STATISTICS', 1, 'Where_index', 2, 'review_scores_rating', 'A', 280, None, None, 'YES', 'BTREE', '', '', 'YES', None)

Executing the query
24 results changed in 0.0187 seconds

Removing created indexes
Indexes removed successfully.


### **4 Data Model Aleterations:**

In [7]:
from sqlalchemy import create_engine, text
import time

username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

schema_and_update_queries = [
    #  columns in LISTINGS table 
    """
    ALTER TABLE LISTINGS 
    ADD COLUMN review_scores_rating FLOAT DEFAULT 0.0,
    ADD COLUMN number_of_reviews INT DEFAULT 0;
    """,

    # update LISTINGS with columns from REVIEWS_STATISTICS table
    """
    UPDATE LISTINGS l
    JOIN REVIEWS_STATISTICS rs ON l.id = rs.listing_id
    SET l.review_scores_rating = rs.review_scores_rating,
        l.number_of_reviews = rs.number_of_reviews;
    """
]

try:
    with engine.connect() as connection:
        for query in schema_and_update_queries:
            start_time = time.time()
            connection.execute(text(query))
            connection.commit()
            end_time = time.time()
            print(f"Query executed successfully in {end_time - start_time:.4f} seconds:\n{query.strip().splitlines()[0]}...")
except Exception as e:
    print(f"An error occurred while executing the schema changes: {e}")
finally:
    print("Schema modifications and data updates completed successfully.")

Query executed successfully in 0.0111 seconds:
ALTER TABLE LISTINGS ...
Query executed successfully in 0.0781 seconds:
UPDATE LISTINGS l...
Schema modifications and data updates completed successfully.


### **4.1 Complex Queries after model update:**

**Complex Query 1: Hosts em New York**

In [8]:
complex_query = """
WITH average_prices AS (
    SELECT 
        l.host_id,
        AVG(l.base_price) AS avg_price
    FROM listings l
    WHERE l.has_availability = TRUE
    GROUP BY l.host_id
),
host_with_high_listings AS (
    SELECT 
        l.host_id,
        COUNT(DISTINCT l.id) AS listings_count
    FROM listings l
    GROUP BY l.host_id
    HAVING COUNT(DISTINCT l.id) > 5
)
SELECT 
    h.id AS host_id,
    h.name AS host_name,
    h.location AS host_location,
    apl.avg_price AS average_price,
    hhl.listings_count AS total_listings
FROM host h
JOIN average_prices apl ON h.id = apl.host_id
JOIN host_with_high_listings hhl ON h.id = hhl.host_id
WHERE apl.avg_price < 150
AND h.location = 'New York, NY'
ORDER BY apl.avg_price ASC;
"""

insert_query = """
INSERT INTO RESULTS (host_id, host_name, host_location, average_price, total_listings)
VALUES (:host_id, :host_name, :host_location, :average_price, :total_listings)
"""

In [9]:
# without index

try:
    with engine.connect() as connection:
        start_time = time.time()

        result = connection.execute(text(complex_query))
        results = result.fetchall()

        if results:
            for row in results:
                connection.execute(
                    text(insert_query),
                    {
                        "host_id": row[0],
                        "host_name": row[1],
                        "host_location": row[2],
                        "average_price": row[3],
                        "total_listings": row[4],
                    }
                )
            
            connection.commit()
            print(f"{len(results)} records inserted into RESULTS.")

            end_time = time.time()
            print(f"{len(results)} results found and inserted in {end_time - start_time:.4f} seconds")
        else:
            print("No results to insert.")

except Exception as e:
    print(f"An error occurred: {e}")

6 records inserted into RESULTS.
6 results found and inserted in 0.0053 seconds


In [10]:
# with indexes and insert functionality

try:
    with engine.connect() as connection:
        print("Existing indexes")
        listings_indexes = connection.execute(text("SHOW INDEX FROM LISTINGS"))
        host_statistics_indexes = connection.execute(text("SHOW INDEX FROM HOST_STATISTICS"))
        host_indexes = connection.execute(text("SHOW INDEX FROM HOST"))
        
        if listings_indexes.rowcount > 0:
            print("\nIndexes in LISTINGS:")
            for row in listings_indexes:
                print(row)
        else:
            print("No indexes in LISTINGS")

        if host_statistics_indexes.rowcount > 0:
            print("\nIndexes in HOST_STATISTICS:")
            for row in host_statistics_indexes:
                print(row)
        else:
            print("No indexes in HOST_STATISTICS")

        if host_indexes.rowcount > 0:
            print("\nIndexes in HOST:")
            for row in host_indexes:
                print(row)
        else:
            print("No indexes in HOST")

        print("\nCreating simple indexes")
        connection.execute(text("CREATE INDEX idx_listings_host_id ON LISTINGS (host_id);")) 
        connection.execute(text("CREATE INDEX idx_host_statistics_host_id ON HOST_STATISTICS (host_id);")) 
        connection.execute(text("CREATE INDEX idx_host_location ON HOST (location(255));")) 
        print("Simple Indexes created successfully.")

        print("\nCreating compound indexes")
        connection.execute(text("CREATE INDEX idx_listings_availability_price ON LISTINGS (has_availability, base_price);")) 
        connection.execute(text("CREATE INDEX idx_host_statistics_listings_count ON HOST_STATISTICS (listings_count);"))
        print("Compound Indexes created successfully")
        
        print("\nExecuting the query")
        start_time = time.time()
        result = connection.execute(text(complex_query))
        results = result.fetchall()

        if results:
            for row in results:
                connection.execute(
                    text(insert_query),
                    {
                        "host_id": row[0],
                        "host_name": row[1],
                        "host_location": row[2],
                        "average_price": row[3],
                        "total_listings": row[4],
                    }
                )
            print(f"{len(results)} records inserted into RESULTS.")

            end_time = time.time()
            print(f"{len(results)} results found and inserted in {end_time - start_time:.4f} seconds")
        else:
            print("No results to insert.")


        print("\nRemoving created indexes")
        connection.execute(text("DROP INDEX idx_listings_host_id ON LISTINGS;"))
        connection.execute(text("DROP INDEX idx_host_statistics_host_id ON HOST_STATISTICS;"))
        connection.execute(text("DROP INDEX idx_host_location ON HOST;"))
        connection.execute(text("DROP INDEX idx_listings_availability_price ON LISTINGS;"))
        connection.execute(text("DROP INDEX idx_host_statistics_listings_count ON HOST_STATISTICS;"))
        print("Indexes removed successfully.")
except Exception as e:
    print(f"An error occurred while creating the indexes or executing the query: {e}")

Existing indexes
No indexes in LISTINGS
No indexes in HOST_STATISTICS
No indexes in HOST

Creating simple indexes
Simple Indexes created successfully.

Creating compound indexes
Compound Indexes created successfully

Executing the query
6 records inserted into RESULTS.
6 results found and inserted in 0.0030 seconds

Removing created indexes
Indexes removed successfully.


In [ ]:
# visualize insert in RESULTS table

username = 'root'
password = 'password'
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

try:
    with engine.connect() as connection:
        query_all = text("SELECT * FROM RESULTS")
        result_all = connection.execute(query_all)
        rows = result_all.fetchall()
        
        print("Contents of the RESULTS table:")
        for row in rows:
            print(row)
        
        query_count = text("SELECT COUNT(*) FROM RESULTS;")
        result_count = connection.execute(query_count)
        count = result_count.scalar()  # single value result
        
        print(f"The RESULTS table contains {count} rows.")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# clear RESULTS table

try:
    with engine.connect() as connection:
        query_clear = text("DELETE FROM RESULTS")
        result_clear = connection.execute(query_clear)
    
        connection.commit()    
        print(f"All rows deleted from the RESULTS table. Total rows affected: {result_clear.rowcount}")

except Exception as e:
    print(f"An error occurred while clearing the RESULTS table: {e}")

**Complex query 2: Atualização de preços de Listings**

In [17]:
new_base_price_update_query = """WITH high_reviewed_listings AS (
    SELECT 
        l.id AS listing_id
    FROM LISTINGS l
    WHERE l.review_scores_rating > 4.5
      AND l.number_of_reviews > 100
),
active_hosts AS (
    SELECT 
        h.id AS host_id,
        MAX(hs.total_listings_count) AS max_listings_count,
        SUM(l.has_availability) AS available_listings
    FROM HOST h
    JOIN HOST_STATISTICS hs ON h.id = hs.host_id
    JOIN LISTINGS l ON h.id = l.host_id
    WHERE h.location = 'Albany, NY'
    GROUP BY h.id
)
UPDATE LISTINGS l
JOIN high_reviewed_listings hl ON l.id = hl.listing_id
JOIN active_hosts ah ON l.host_id = ah.host_id
SET l.base_price = l.base_price * 1.1
WHERE ah.max_listings_count > 5
  AND ah.available_listings > 3
  AND l.base_price < 300;
"""

In [31]:
# without index

username = 'root'
password = 'password'
# password = '1234' 
host = 'localhost'
port = '3306'
database_name = 'Project_DB'

db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}"
engine = create_engine(db_url)

try:
    with engine.connect() as connection:
        start_time = time.time()
        result = connection.execute(text(new_base_price_update_query))
        connection.commit()
        end_time = time.time()
        
        affected_rows = result.rowcount
        print(f"{affected_rows} results changed in {end_time - start_time:.4f} seconds")
except Exception as e:
    print(f"An error occurred while executing the query: {e}")

24 results changed in 0.0502 seconds


In [33]:
try:
    with engine.connect() as connection:
        print("Existing indexes:")
        listings_indexes = connection.execute(text("SHOW INDEX FROM LISTINGS"))

        if listings_indexes.rowcount > 0:
            print("\nIndexes in LISTINGS:")
            for row in listings_indexes:
                print(row)
        else:
            print("No indexes in LISTINGS")

        # add indexes
        print("\nCreating indexes...")
        connection.execute(text("CREATE INDEX id_index ON LISTINGS (id);"))
        connection.execute(text("CREATE INDEX list_index ON LISTINGS (base_price, has_availability);"))
        connection.execute(text("CREATE INDEX review_scores_index ON LISTINGS (review_scores_rating, number_of_reviews);"))
        print("Indexes created successfully.")

        print("\nExecuting the query...")
        start_time = time.time()
        result = connection.execute(text(new_base_price_update_query))
        connection.commit()
        end_time = time.time()
        
        affected_rows = result.rowcount
        print(f"{affected_rows} results changed in {end_time - start_time:.4f} seconds")

        # remove indexes
        print("\nRemoving created indexes...")
        connection.execute(text("DROP INDEX id_index ON LISTINGS;"))
        connection.execute(text("DROP INDEX list_index ON LISTINGS;"))
        connection.execute(text("DROP INDEX review_scores_index ON LISTINGS;"))
        print("Indexes removed successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

Existing indexes:
No indexes in LISTINGS

Creating indexes...
Indexes created successfully.

Executing the query...
20 results changed in 0.0168 seconds

Removing created indexes...
Indexes removed successfully.
